## 1. Introduction

### 1.1 작업 환경 설정

In [2]:
# 패키지 로드
library(tidyverse) # dplyr, ggplot2 등을 포함하는 패키지들의 패키지
library(data.table) # 데이터 핸들링

# 작업 디렉토리 설정
setwd('C:/Users/supre.000/Desktop/elo-merchant-category-recommendation')

Warning message:
"package 'tidyverse' was built under R version 3.4.4"-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 2.2.1     √ purrr   0.2.4
√ tibble  1.4.2     √ dplyr   0.7.4
√ tidyr   0.8.1     √ stringr 1.2.0
√ readr   1.1.1     √ forcats 0.3.0
Warning message:
"package 'tibble' was built under R version 3.4.4"Warning message:
"package 'tidyr' was built under R version 3.4.4"Warning message:
"package 'forcats' was built under R version 3.4.4"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

The following object is masked from 'package:purrr':

    transpose



In [3]:
# 파일 로드 (1)
trans <- fread('historical_transactions.csv') # 크기(2.65GB)가 크므로, 파일 불러올 때 주의!

Read 29112361 rows and 14 (of 14) columns from 2.650 GB file in 00:01:36


In [4]:
# 파일 로드 (2)
merchants <- fread('merchants.csv')
n_merchants <- fread('new_merchant_transactions.csv')
train <- fread('train.csv')
test <- fread('test.csv')
sub <- fread('sample_submission.csv')

Read 1963031 rows and 14 (of 14) columns from 0.177 GB file in 00:00:05


Elo 경진대회에서 제공하는 파일은 총 6가지로, 각각은 다음에 관한 데이터다.

- `train.csv`: 훈련 데이터(training set)
- `test.csv`: 시험 데이터(test set)
- `sample_submission.csv`: 제출 파일 양식 예시
- `historical_transactions.csv`: 각 `card_id`별 3개월 간 거래 데이터
- `merchants.csv`: 상점 고유번호(`merchant_id`) 기준으로 작성된 상점 관련 메타 데이터
- `new_merchant_transactions.csv`: 거래 데이터에 포함되지 않은 상점들에서 발생한 2개월 간 거래 데이터

Elo 경진대회는 **카드별 충성도 점수를 보다 정확히 예측하는 것**을 목표로 한다.  먼저 제출해야 하는 파일을 살펴보자.

In [5]:
# 제출 양식 구조 확인
str(sub)

# 충성도 점수를 예측해야 하는 카드 개수 
sub %>%
    n_distinct('card_id')

Classes 'data.table' and 'data.frame':	123623 obs. of  2 variables:
 $ card_id: chr  "C_ID_0ab67a22ab" "C_ID_130fd0cbdd" "C_ID_b709037bc5" "C_ID_d27d835a9f" ...
 $ target : int  0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, ".internal.selfref")=<externalptr> 


[1] 123623

확인 결과, 123,623개 카드 고유번호의 충성도 점수를 예측해야 하는 문제로 나타났다.

### 1.2 데이터 구조 확인

본격적인 데이터 탐색에 앞서 전반적인 데이터 구조를 확인해보자.

#### 1.2.1 훈련 데이터(`train.csv`)

훈련 데이터에는 총 6개의 변수가 포함되어 있다.

- `first_active_month`: 첫 구매가 이루어진 연-월 정보(YYYY-MM)
- `card_id`: 카드 고유번호
- `feature_1`: 익명화 처리된 카드 관련 범주형 변수 1
- `feature_2`: 익명화 처리된 카드 관련 범주형 변수 2
- `feature_3`: 익명화 처리된 카드 관련 범주형 변수 3
- `target`: 충성도 점수

In [6]:
str(train)

Classes 'data.table' and 'data.frame':	201917 obs. of  6 variables:
 $ first_active_month: chr  "2017-06" "2017-01" "2016-08" "2017-09" ...
 $ card_id           : chr  "C_ID_92a2005557" "C_ID_3d0044924f" "C_ID_d639edf6cd" "C_ID_186d6a6901" ...
 $ feature_1         : int  5 4 2 4 1 4 3 3 2 2 ...
 $ feature_2         : int  2 1 2 3 3 2 2 2 1 2 ...
 $ feature_3         : int  1 0 0 0 0 0 1 1 0 0 ...
 $ target            : num  -0.82 0.393 0.688 0.142 -0.16 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [7]:
head(train)

first_active_month,card_id,feature_1,feature_2,feature_3,target
2017-06,C_ID_92a2005557,5,2,1,-0.8202826
2017-01,C_ID_3d0044924f,4,1,0,0.3929132
2016-08,C_ID_d639edf6cd,2,2,0,0.6880560
2017-09,C_ID_186d6a6901,4,3,0,0.1424952
2017-11,C_ID_cdbd2c0db2,1,3,0,-0.1597492
2016-09,C_ID_0894217f2f,4,2,0,0.8715853


#### 1.2.2 시험 데이터(`test.csv`)

시험 데이터에는 훈련 데이터와 비교했을 때, `target` 변수를 제외한 나머지 다섯 개 변수가 포함되어 있다.

In [8]:
str(test)

Classes 'data.table' and 'data.frame':	123623 obs. of  5 variables:
 $ first_active_month: chr  "2017-04" "2017-01" "2017-08" "2017-12" ...
 $ card_id           : chr  "C_ID_0ab67a22ab" "C_ID_130fd0cbdd" "C_ID_b709037bc5" "C_ID_d27d835a9f" ...
 $ feature_1         : int  3 2 5 2 5 5 1 3 4 5 ...
 $ feature_2         : int  3 3 1 1 1 1 2 1 3 1 ...
 $ feature_3         : int  1 0 1 0 1 1 0 1 0 1 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [9]:
head(test)

first_active_month,card_id,feature_1,feature_2,feature_3
2017-04,C_ID_0ab67a22ab,3,3,1
2017-01,C_ID_130fd0cbdd,2,3,0
2017-08,C_ID_b709037bc5,5,1,1
2017-12,C_ID_d27d835a9f,2,1,0
2015-12,C_ID_2b5e3df5c2,5,1,1
2017-07,C_ID_5814b4f13c,5,1,1


#### 1.2.3 거래 데이터(`historical_transactions.csv`)

거래 데이터에는 총 14개의 변수가 포함되어 있다.

- `authorized_flag`: 구매 승인 여부(승인된 경우 'Y', 거절된 경우 'N')
- `card_id`: 카드 고유번호
- `state_id`: 국가(state)? 혹은 주(state) 고유번호
- `city_id`: 익명화 처리된 도시 고유번호
- `category_1`: 익명화 처리된 범주 1
- `category_2`: 익명화 처리된 범주 2
- `category_3`: 익명화 처리된 범주 3
- `installments`: 할부 개월 수
- `merchant_category_id`: 상점 분류 카테고리 고유번호
- `subsector_id`: 상점 분류 그룹 카테고리 고유번호
- `month_lag`: 기준일 시점으로부터 경과한 개월 수
- `purchase_amount`: 표준화된 구매 금액
- `purchase_date`: 구매 일자

In [10]:
str(trans)

Classes 'data.table' and 'data.frame':	29112361 obs. of  14 variables:
 $ authorized_flag     : chr  "Y" "Y" "Y" "Y" ...
 $ card_id             : chr  "C_ID_4e6213e9bc" "C_ID_4e6213e9bc" "C_ID_4e6213e9bc" "C_ID_4e6213e9bc" ...
 $ city_id             : int  88 88 88 88 88 333 88 3 88 88 ...
 $ category_1          : chr  "N" "N" "N" "N" ...
 $ installments        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ category_3          : chr  "A" "A" "A" "A" ...
 $ merchant_category_id: int  80 367 80 560 80 80 278 80 80 80 ...
 $ merchant_id         : chr  "M_ID_e020e9b302" "M_ID_86ec983688" "M_ID_979ed661fc" "M_ID_e6d5ae8ea6" ...
 $ month_lag           : int  -8 -7 -6 -5 -11 0 -11 -3 -8 -11 ...
 $ purchase_amount     : num  -0.703 -0.733 -0.72 -0.735 -0.723 ...
 $ purchase_date       : chr  "2017-06-25 15:33:07" "2017-07-15 12:10:45" "2017-08-09 22:04:29" "2017-09-02 10:06:26" ...
 $ category_2          : num  1 1 1 1 1 1 1 1 1 1 ...
 $ state_id            : int  16 16 16 16 16 9 16 16 16 16 ...
 $ subsec

In [11]:
head(trans)

authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.7033309,2017-06-25 15:33:07,1,16,37
Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.7331285,2017-07-15 12:10:45,1,16,16
Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.7203860,2017-08-09 22:04:29,1,16,37
Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.7353524,2017-09-02 10:06:26,1,16,34
Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.7228654,2017-03-10 01:14:19,1,16,37
Y,C_ID_4e6213e9bc,333,N,0,A,80,M_ID_50af771f8d,0,-0.7348866,2018-02-24 08:45:05,1,9,37


~~`month_lag` 를 확인한 결과, 기준 연-월(reference date)은 2018년 2월로 보인다.~~

이는 `card_id`가 'C_ID_4e6213e9bc'인 경우에만 해당되는 값일 수 있으므로, 다른 `card_id`를 찾아 reference date을 살펴보자.

In [12]:
tail(trans)

authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
Y,C_ID_e49b1996b0,11,N,1,B,130,M_ID_ea5d727248,0,-0.6943150,2018-02-07 18:17:01,2,23,41
Y,C_ID_2863d2fa95,-1,Y,1,B,360,M_ID_edd92b6720,-1,-0.6327063,2017-01-20 08:52:04,NA,-1,34
Y,C_ID_2863d2fa95,-1,Y,1,B,360,M_ID_edd92b6720,0,-0.6327063,2017-02-20 04:40:50,NA,-1,34
Y,C_ID_5c240d6e3c,3,N,0,A,278,M_ID_9cdcfe8673,0,-0.6577405,2017-12-26 18:37:51,1,16,37
Y,C_ID_5c240d6e3c,331,N,0,A,514,M_ID_1a75f94f92,-1,0.7706196,2017-11-24 14:18:15,1,16,9
Y,C_ID_5c240d6e3c,331,N,0,A,514,M_ID_1a75f94f92,-2,1.1344115,2017-10-26 14:09:40,1,16,9


각 카드별로 `month_lag`가 0인 연-월이 상이해보인다. 이러한 가설이 맞는지 `month_lag`가 0인 것만 몇 개 추출해보자.

In [34]:
trans %>%
    filter(month_lag == 0) %>% # month_lag가 0 인 행만을 걸러낸 후,
    distinct(card_id, .keep_all = TRUE) %>% # cf) .keep_all = TRUE를 사용하면 나머지 컬럼을 전부 보여줌. 
    head(30) # 상위 30 행만을 출력

authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
Y,C_ID_4e6213e9bc,333,N,0,A,80,M_ID_50af771f8d,0,-0.7348866,2018-02-24 08:45:05,1,9,37
Y,C_ID_5037ff576e,-1,N,1,B,307,M_ID_949996e6b5,0,-0.6717752,2017-12-24 19:28:09,NA,-1,19
Y,C_ID_0e171c1b48,277,N,0,A,80,M_ID_ad5e2702ac,0,-0.7318813,2018-02-16 21:04:55,4,13,37
Y,C_ID_48fb13e70f,69,N,3,C,367,M_ID_3f3084da59,0,-0.3922220,2018-02-20 15:33:35,1,9,16
Y,C_ID_fc8e41b9cf,85,N,1,B,605,M_ID_c460757574,0,-0.6266957,2018-02-21 05:47:19,3,8,2
Y,C_ID_b02e55aa9d,283,N,0,A,560,M_ID_e5e444c54a,0,-0.7162537,2018-02-12 16:24:35,1,9,34
Y,C_ID_b271e7ab60,17,N,0,A,307,M_ID_b54d454927,0,-0.4830873,2018-02-12 11:31:53,4,22,19
Y,C_ID_4bed29d75c,272,N,1,B,695,M_ID_7f41df8893,0,-0.7251194,2018-01-23 14:50:12,1,9,34
Y,C_ID_93d201f55f,193,N,0,A,705,M_ID_b9e5c109f4,0,0.1058167,2018-02-17 09:51:19,1,15,33
Y,C_ID_3fff3df454,30,N,0,A,683,M_ID_e053bbe5fb,0,-0.6981468,2018-02-18 15:41:00,3,17,34


30개를 추출하여 확인한 결과, 예상대로 각 카드별로 기준 연-월은 상이한 것으로 나타났다. 

#### 1.2.4 상점 관련 메타 데이터(merchants.csv)

상점 관련 메타 데이터



## 2. EDA

### 2.1 충성도 점수의 분포는 어떠한가?

### 2.2


### 2.3